<h1> Notebook to analyse Model Drift and enable continuous Model Monitoring </h1>

In [45]:
import numpy as np
import pandas as pd
import os

In [57]:
#Dataset details
df_marketing_campaign = pd.read_csv(r"/kaggle/input/customer-personality-analysis/marketing_campaign.csv", sep="\t")

In [3]:
df_marketing_campaign.head()

,ID,Year_Birth,Education,Marital_Status,Income,Kidhome,Teenhome,Dt_Customer,Recency,MntWines,...,NumWebVisitsMonth,AcceptedCmp3,AcceptedCmp4,AcceptedCmp5,AcceptedCmp1,AcceptedCmp2,Complain,Z_CostContact,Z_Revenue,Response
0,5524,1957,Graduation,Single,58138.0,0,0,04-09-2012,58,635,...,7,0,0,0,0,0,0,3,11,1
1,2174,1954,Graduation,Single,46344.0,1,1,08-03-2014,38,11,...,5,0,0,0,0,0,0,3,11,0
2,4141,1965,Graduation,Together,71613.0,0,0,21-08-2013,26,426,...,4,0,0,0,0,0,0,3,11,0
3,6182,1984,Graduation,Together,26646.0,1,0,10-02-2014,26,11,...,6,0,0,0,0,0,0,3,11,0
4,5324,1981,PhD,Married,58293.0,1,0,19-01-2014,94,173,...,5,0,0,0,0,0,0,3,11,0


In [4]:
df_marketing_campaign.columns

Index(['ID', 'Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome',
       'Teenhome', 'Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth',
       'AcceptedCmp3', 'AcceptedCmp4', 'AcceptedCmp5', 'AcceptedCmp1',
       'AcceptedCmp2', 'Complain', 'Z_CostContact', 'Z_Revenue', 'Response'],
      dtype='object')

In [5]:
df_marketing_campaign['Marital_Status'].value_counts()

Married     864
Together    580
Single      480
Divorced    232
Widow        77
Alone         3
Absurd        2
YOLO          2
Name: Marital_Status, dtype: int64

In [6]:
#very in-frequent categories, removing for proper/stable chi-square statistics
df_marketing_campaign = df_marketing_campaign[~df_marketing_campaign.isin(['Alone', 'Absurd','YOLO'])]

In [7]:
cols = ['Year_Birth', 'Education', 'Marital_Status', 'Income', 'Kidhome','Dt_Customer', 'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']


df_marketing_campaign = df_marketing_campaign[cols]


In [8]:
df_marketing_campaign.head()

,Year_Birth,Education,Marital_Status,Income,Kidhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth
0,1957,Graduation,Single,58138.0,0,04-09-2012,58,635,88,546,172,88,88,3,8,10,4,7
1,1954,Graduation,Single,46344.0,1,08-03-2014,38,11,1,6,2,1,6,2,1,1,2,5
2,1965,Graduation,Together,71613.0,0,21-08-2013,26,426,49,127,111,21,42,1,8,2,10,4
3,1984,Graduation,Together,26646.0,1,10-02-2014,26,11,4,20,10,3,5,2,2,0,4,6
4,1981,PhD,Married,58293.0,1,19-01-2014,94,173,43,118,46,27,15,5,5,3,6,5


In [9]:
data_1 = df_marketing_campaign.sample(frac=0.5) # creating samples
data_2 = df_marketing_campaign.sample(frac=0.5)

In [10]:
display(data_1.head())
print('\n\n')
display(data_2.head())

,Year_Birth,Education,Marital_Status,Income,Kidhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth
1273,1966,Master,Married,57183.0,1,19-03-2013,51,464,5,64,7,0,70,8,9,1,7,8
119,1969,Graduation,Together,21918.0,1,14-11-2013,37,1,6,7,11,5,3,2,2,0,3,6
406,1972,PhD,Single,27213.0,1,30-08-2012,19,19,3,26,10,9,3,3,2,0,4,8
772,1965,Graduation,Single,42720.0,1,24-04-2013,9,392,5,91,28,26,112,8,7,3,7,8
2021,1982,Master,Married,43815.0,1,05-04-2013,53,14,10,29,4,7,15,1,2,0,3,6


,Year_Birth,Education,Marital_Status,Income,Kidhome,Dt_Customer,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth
1903,1971,Graduation,Divorced,29819.0,1,24-10-2013,77,9,1,24,2,1,2,3,3,0,3,6
1796,1955,Graduation,Single,62535.0,0,03-10-2013,13,163,48,90,0,45,52,1,3,2,8,3
183,1984,Master,Married,30477.0,1,22-01-2014,16,47,1,6,0,0,12,1,1,1,3,6
1950,1940,PhD,Single,51141.0,0,08-07-2013,96,144,0,7,0,0,6,1,3,1,4,5
1866,1968,Master,Together,32889.0,1,06-09-2013,78,28,0,13,4,2,8,3,2,1,3,5


<h2> Kolmogorov-Smirnov Test </h2>

Kolmogorov-Smirnov (K-S) test for detecting whether the two samples are identical. for numeric data only

In [11]:
'''
 `two-sided`: H0 is that the two distributions are
  identical, F(x)=G(x) for all x;
  H1 is that they are not identical.
  
'''
from scipy import stats
import random

numeric_cols = ['Income',  'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']

#Significance level

p_value = 0.05
rejected = 0

for col in numeric_cols:
    test = stats.ks_2samp(data_1[col], data_2[col])

    print(col, test)
    if test[1] < p_value:
        rejected += 1
        print("Column rejected", col)

print("We rejected ",rejected," columns in total out of {} columns".format(len(numeric_cols)))


Income KstestResult(statistic=0.026785714285714284, pvalue=0.8167638092508208)
Recency KstestResult(statistic=0.04107142857142857, pvalue=0.3014146770679847)
MntWines KstestResult(statistic=0.038392857142857145, pvalue=0.3811835212209135)
MntFruits KstestResult(statistic=0.019642857142857142, pvalue=0.9822154133190888)
MntMeatProducts KstestResult(statistic=0.0375, pvalue=0.4104935563287252)
MntFishProducts KstestResult(statistic=0.014285714285714285, pvalue=0.9998505708532414)
MntSweetProducts KstestResult(statistic=0.022321428571428572, pvalue=0.9431423916136951)
MntGoldProds KstestResult(statistic=0.04196428571428571, pvalue=0.27760582310123827)
NumDealsPurchases KstestResult(statistic=0.009821428571428571, pvalue=0.9999999987970982)
NumWebPurchases KstestResult(statistic=0.033035714285714286, pvalue=0.5742817944592722)
NumCatalogPurchases KstestResult(statistic=0.025, pvalue=0.8754126208113381)
NumStorePurchases KstestResult(statistic=0.030357142857142857, pvalue=0.6806946185920665

Let's assume the income of people increased by 15-20% in future.

In [12]:
data_2_future = data_2.copy()
pct_low = 0.15
pct_high = 0.20
data_2_future['Income'] = data_2_future['Income'].apply(lambda v : v * (1 + random.uniform(pct_low,pct_high)))

In [13]:
data_2.describe()

,Year_Birth,Income,Kidhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth
count,1120.000000,1107.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000
mean,1968.295536,52282.691960,0.441071,48.241964,299.936607,26.719643,175.889286,37.960714,28.609821,43.768750,2.246429,4.057143,2.683036,5.776786,5.221429
std,12.332277,21613.450645,0.538185,29.255819,334.518830,40.684030,235.186706,54.931481,43.437818,51.090688,1.884509,2.780364,2.933045,3.239915,2.422173
min,1899.000000,1730.000000,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1958.000000,34976.000000,0.000000,23.000000,23.000000,2.000000,16.000000,3.000000,1.000000,9.000000,1.000000,2.000000,0.000000,3.000000,3.000000
50%,1970.000000,51651.000000,0.000000,48.000000,171.000000,8.000000,68.000000,12.000000,9.000000,25.000000,2.000000,4.000000,2.000000,5.000000,6.000000
75%,1977.000000,69737.000000,1.000000,74.000000,492.000000,32.000000,247.500000,50.250000,35.250000,57.250000,3.000000,6.000000,4.000000,8.000000,7.000000
max,1996.000000,160803.000000,2.000000,99.000000,1486.000000,199.000000,1725.000000,258.000000,263.000000,362.000000,15.000000,27.000000,28.000000,13.000000,20.000000


In [14]:
data_2_future.describe()

,Year_Birth,Income,Kidhome,Recency,MntWines,MntFruits,MntMeatProducts,MntFishProducts,MntSweetProducts,MntGoldProds,NumDealsPurchases,NumWebPurchases,NumCatalogPurchases,NumStorePurchases,NumWebVisitsMonth
count,1120.000000,1107.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000,1120.000000
mean,1968.295536,60107.201512,0.441071,48.241964,299.936607,26.719643,175.889286,37.960714,28.609821,43.768750,2.246429,4.057143,2.683036,5.776786,5.221429
std,12.332277,24894.180235,0.538185,29.255819,334.518830,40.684030,235.186706,54.931481,43.437818,51.090688,1.884509,2.780364,2.933045,3.239915,2.422173
min,1899.000000,1962.936010,0.000000,0.000000,0.000000,0.000000,1.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000,0.000000
25%,1958.000000,40348.255988,0.000000,23.000000,23.000000,2.000000,16.000000,3.000000,1.000000,9.000000,1.000000,2.000000,0.000000,3.000000,3.000000
50%,1970.000000,59898.160864,0.000000,48.000000,171.000000,8.000000,68.000000,12.000000,9.000000,25.000000,2.000000,4.000000,2.000000,5.000000,6.000000
75%,1977.000000,79944.749110,1.000000,74.000000,492.000000,32.000000,247.500000,50.250000,35.250000,57.250000,3.000000,6.000000,4.000000,8.000000,7.000000
max,1996.000000,178377.958795,2.000000,99.000000,1486.000000,199.000000,1725.000000,258.000000,263.000000,362.000000,15.000000,27.000000,28.000000,13.000000,20.000000


In [15]:
'''
 `two-sided`: The null hypothesis is that the two distributions are
  identical, F(x)=G(x) for all x; the alternative is that they are not
  identical.
  
'''
from scipy import stats
import random

numeric_cols = ['Income',  'Recency', 'MntWines', 'MntFruits',
       'MntMeatProducts', 'MntFishProducts', 'MntSweetProducts',
       'MntGoldProds', 'NumDealsPurchases', 'NumWebPurchases',
       'NumCatalogPurchases', 'NumStorePurchases', 'NumWebVisitsMonth']

#Significance level

p_value = 0.05
rejected = 0

for col in numeric_cols:
    test = stats.ks_2samp(data_1[col], data_2_future[col])

    print(col, test)
    if test[1] < p_value:
        rejected += 1
        print("Column rejected", col)

print("We rejected ",rejected," columns in total out of {} columns".format(len(numeric_cols)))


Income KstestResult(statistic=0.16160714285714287, pvalue=3.526887911398711e-13)
Column rejected Income
Recency KstestResult(statistic=0.04107142857142857, pvalue=0.3014146770679847)
MntWines KstestResult(statistic=0.038392857142857145, pvalue=0.3811835212209135)
MntFruits KstestResult(statistic=0.019642857142857142, pvalue=0.9822154133190888)
MntMeatProducts KstestResult(statistic=0.0375, pvalue=0.4104935563287252)
MntFishProducts KstestResult(statistic=0.014285714285714285, pvalue=0.9998505708532414)
MntSweetProducts KstestResult(statistic=0.022321428571428572, pvalue=0.9431423916136951)
MntGoldProds KstestResult(statistic=0.04196428571428571, pvalue=0.27760582310123827)
NumDealsPurchases KstestResult(statistic=0.009821428571428571, pvalue=0.9999999987970982)
NumWebPurchases KstestResult(statistic=0.033035714285714286, pvalue=0.5742817944592722)
NumCatalogPurchases KstestResult(statistic=0.025, pvalue=0.8754126208113381)
NumStorePurchases KstestResult(statistic=0.030357142857142857, 

### There is a detected data-drift in Income column as pvalue < acceptance threshold (0.05)

<h2> Population Stability Index </h2>

### Population Stability Index: PSI is a symmetric version of KL Divergence. It's used a lot in financial domain and below are some widely accepted threshold for it. It's used for detecting the data-drift in numerical columns.<br><br>

#### a) When PSI<=0.1
##### This means there is no change or shift in the distributions of both datasets.

#### b) 0.1< PSI<0.2
##### This indicates a slight change or shift has occurred.
                
#### c) PSI>0.2
##### This indicates a large shift in the distribution has occurred between both datasets.

In [16]:
import numpy as np

def calculate_psi(expected, actual, buckettype='bins', buckets=10, axis=0):
    '''Calculate the PSI (population stability index) across all variables
    Args:
       expected: numpy matrix of original values
       actual: numpy matrix of new values, same size as expected
       buckettype: type of strategy for creating buckets, bins splits into even splits, quantiles splits into quantile buckets
       buckets: number of quantiles to use in bucketing variables
       axis: axis by which variables are defined, 0 for vertical, 1 for horizontal
    Returns:
       psi_values: ndarray of psi values for each variable
    '''

    def psi(expected_array, actual_array, buckets):
        '''Calculate the PSI for a single variable
        Args:
           expected_array: numpy array of original values
           actual_array: numpy array of new values, same size as expected
           buckets: number of percentile ranges to bucket the values into
        Returns:
           psi_value: calculated PSI value
        '''

        def scale_range (input, min, max):
            input += -(np.min(input))
            input /= np.max(input) / (max - min)
            input += min
            return input


        breakpoints = np.arange(0, buckets + 1) / (buckets) * 100

        if buckettype == 'bins':
            breakpoints = scale_range(breakpoints, np.min(expected_array), np.max(expected_array))
        elif buckettype == 'quantiles':
            breakpoints = np.stack([np.percentile(expected_array, b) for b in breakpoints])



        expected_percents = np.histogram(expected_array, breakpoints)[0] / len(expected_array)
        actual_percents = np.histogram(actual_array, breakpoints)[0] / len(actual_array)

        def sub_psi(e_perc, a_perc):
            '''Calculate the actual PSI value from comparing the values.
               Update the actual value to a very small number if equal to zero
            '''
            if a_perc == 0:
                a_perc = 0.0001
            if e_perc == 0:
                e_perc = 0.0001

            value = (e_perc - a_perc) * np.log(e_perc / a_perc)
            return(value)

        psi_value = np.sum(sub_psi(expected_percents[i], actual_percents[i]) for i in range(0, len(expected_percents)))

        return(psi_value)

    if len(expected.shape) == 1:
        psi_values = np.empty(len(expected.shape))
    else:
        psi_values = np.empty(expected.shape[axis])

    for i in range(0, len(psi_values)):
        if len(psi_values) == 1:
            psi_values = psi(expected, actual, buckets)
        elif axis == 0:
            psi_values[i] = psi(expected[:,i], actual[:,i], buckets)
        elif axis == 1:
            psi_values[i] = psi(expected[i,:], actual[i,:], buckets)

    return(psi_values)

In [17]:
for col in numeric_cols:
    val = calculate_psi(data_1[col], data_2[col])
    if val <  0.1:
        print(col, val, "No significant drift/shift detected")
    elif val > 0.1 and val < 0.2:
        print(col, val, "Slight drift/shift detected")
    else:
        print(col, val, "Significant drift/shift detected")

Income 0.0025448323243239057 No significant drift/shift detected
Recency 0.006118677483977404 No significant drift/shift detected
MntWines 0.017843783264784894 No significant drift/shift detected
MntFruits 0.003658459913827903 No significant drift/shift detected
MntMeatProducts 0.0084433398359502 No significant drift/shift detected
MntFishProducts 0.0035194751576293594 No significant drift/shift detected
MntSweetProducts 0.00734275446062029 No significant drift/shift detected
MntGoldProds 0.004621466060646782 No significant drift/shift detected
NumDealsPurchases 0.017888008751334703 No significant drift/shift detected
NumWebPurchases 0.015339794651098764 No significant drift/shift detected
NumCatalogPurchases 0.002004102730427509 No significant drift/shift detected
NumStorePurchases 0.007398713856465929 No significant drift/shift detected
NumWebVisitsMonth 0.010242579766239846 No significant drift/shift detected


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


In [18]:
for col in numeric_cols:
    val = calculate_psi(data_1[col], data_2_future[col])
    if val <  0.1:
        print(col, val, "No significant drift/shift detected")
    elif val > 0.1 and val < 0.2:
        print(col, val, "Slight drift/shift detected")
    else:
        print(col, val, "Significant drift/shift detected")

Income 0.08492730078142666 No significant drift/shift detected
Recency 0.006118677483977404 No significant drift/shift detected
MntWines 0.017843783264784894 No significant drift/shift detected
MntFruits 0.003658459913827903 No significant drift/shift detected
MntMeatProducts 0.0084433398359502 No significant drift/shift detected
MntFishProducts 0.0035194751576293594 No significant drift/shift detected
MntSweetProducts 0.00734275446062029 No significant drift/shift detected
MntGoldProds 0.004621466060646782 No significant drift/shift detected
NumDealsPurchases 0.017888008751334703 No significant drift/shift detected
NumWebPurchases 0.015339794651098764 No significant drift/shift detected
NumCatalogPurchases 0.002004102730427509 No significant drift/shift detected
NumStorePurchases 0.007398713856465929 No significant drift/shift detected
NumWebVisitsMonth 0.010242579766239846 No significant drift/shift detected


/opt/conda/lib/python3.7/site-packages/ipykernel_launcher.py:60: DeprecationWarning: Calling np.sum(generator) is deprecated, and in the future will give a different result. Use np.sum(np.fromiter(generator)) or the python sum builtin instead.


### Chi-square test for homogeneity for categorical data 

In [19]:
from scipy.stats import chi2_contingency #

def drift_chisquare(sample1, sample2):
    return chi2_contingency([sample1, sample2])[1]

In [20]:
#Let's look at categorcial variables
categorical_cols = ['Education', 'Marital_Status',]

for col in categorical_cols:
    print(col, set(df_marketing_campaign[col].values))

Education {'Graduation', 'Master', 'Basic', 'PhD', '2n Cycle'}
Marital_Status {nan, 'Widow', 'Divorced', 'Single', 'Married', 'Together'}


In [21]:
for col in categorical_cols:
    val = drift_chisquare(data_1[col].value_counts(),data_2[col].value_counts() )
    
    print(col,val)
    rejected = 0
    if val < p_value:
        rejected += 1
        print("Column rejected", col)

print("We rejected ",rejected," columns in total out of {} columns".format(len(categorical_cols)))


Education 0.8923631288999275
Marital_Status 0.5165356475482418
We rejected  0  columns in total out of 2 columns


### assume married population increased by 25% in future.

In [22]:
def future_marital_status(v):
    if v == "Married":
        return v
    else:
        coin_toss = random.uniform(0,1)
        if coin_toss > 0.25:
            return v
        return 'Married'
    
data_2_future['Marital_Status'] = data_2_future['Marital_Status'].apply(lambda v : future_marital_status(v))

In [23]:
data_2['Marital_Status'].value_counts(), data_2_future['Marital_Status'].value_counts()

(Married     467
 Together    257
 Single      238
 Divorced    114
 Widow        40
 Name: Marital_Status, dtype: int64,
 Married     551
 Together    225
 Single      205
 Divorced    101
 Widow        35
 Name: Marital_Status, dtype: int64)

In [24]:
for col in categorical_cols:
    val = drift_chisquare(data_1[col].value_counts(),data_2_future[col].value_counts() )
    print(col,val)
    rejected = 0
    if val < p_value:
        rejected += 1
        print("Column rejected", col)

print("We rejected ",rejected," columns in total out of {} columns".format(len(categorical_cols)))


Education 0.8923631288999275
Marital_Status 2.1820656379674705e-05
Column rejected Marital_Status
We rejected  1  columns in total out of 2 columns


## END